## Save a Copy This Notebook to Google Drive

(Ignore this step if you already did this step and are using the notebook stored in your Drive)

Save your own copy of the notebook in your Google Drive with the "Copy to Drive" button near the top left of the notebook. A copy of this notebook named "Copy of 03_everyday_use" will be saved to a folder called "Colab Notebooks" on your Drive. Now as you update the code, the changes will be saved. Otherwise, they would reset every time you closed the notebook. 

You can confirm you are using the copy of the notebook in your Drive by looking for the "Copy to Drive" button. If it's gone, you're in the right notebook.

## Startup

Colab's computers don't stay on all the time, so each time you open up this notebook you need to connect to Google Drive again and download all the Concha code. The next two cells get those working.

In [ ]:
# Connect to Google Drive
from google.colab import drive
drive.mount('/content/drive')
# Once you copy the code into the text box below, type "Enter" to finish the connnection.

In [1]:
# Bring in the Concha code
!pip -q install concha
from concha.planner import Planner
from concha.importers import Square
from concha.weather import NOAA

## Make a new Planner
A "Planner" is what makes predictions for each product at a particular location. Once making a planner you can assign a Square importer and a weather importer.

In [2]:
# First let's see what locations are available.
sq = Square()
locs = sq.get_locations()
locs

{'Center City': {'id': 'L074SHENBT3WK',
  'name': 'Center City',
  'address': '275 S Raymond Ave, CA, US, Pasadena, 91105',
  'lat': 34.141888,
  'lng': -118.149274,
  'timezone': 'America/Los_Angeles'},
 'Concha West': {'id': 'LPZBW2H2WPS6M',
  'name': 'Concha West',
  'address': '415 S Arroyo Blvd, CA, US, Pasadena, 91105',
  'lat': 34.13827779625123,
  'lng': -118.1661243381397,
  'timezone': 'America/Los_Angeles'}}

In [3]:
#@title New Planner Setup

planner_name = "cafe_1" #@param {type:"string"}
location_name = "" #@param {type:"string"}

# Add the sales history importer
planner = Planner(planner_name=planner_name)
planner.set_importer(sq)
planner.set_location(location_name)

# Add the weather importer
weather = NOAA()
planner.set_weather_agent(weather)
# This uses the lat/lng from the location above to find the nearest weather station.
planner.set_weather_station()

location for cafe_1 set to Center City.


## Run a planner

For day-to-day operations the steps are:

1.) Pull in the most recent sales data from Square.

2.) Train the machine learning models for each product.

3.) Make predictions.

In [4]:
# Fire up the planner
planner_name = "cafe_1" #@param {type:"string"}
planner = Planner(planner_name=planner_name)

# Uncomment (delete the "#") these two lines below to limit which products to predict.
# products_list = ['product-1', 'product-2'] #@param {type:"raw"}
# planner.update_history(products=products_list)

# Get the most recent history
new_transactions = planner.update_history()
new_transactions.head()

# Skip to the Updating the Products if you haven't done that yet step.

Importing from: /home/thomas/concha_planners/cafe_1/history
Imported csv columns: timestamp, product, quantity


,timestamp,product,quantity
0,2020-10-20T02:24:03Z,Basic - Glazed,4.0


In [9]:
# Train the models
planner.train()

#Make a prediction for the best amount to make for each product
predictions = planner.predict()
predictions.head()

Training model for product: Basic - Glazed
Training model for product: Basic - Maple Bar
Training model for product: Basic - Chocolate
Training model for product: 4
Training model for product: 5
Predicting production for product: Basic - Glazed
Predicting production for product: Basic - Maple Bar
Predicting production for product: Basic - Chocolate
Predicting production for product: 4
Predicting production for product: 5


,date,product,day_of_week,tmin,tmax,prcp,production
0,2020-11-19,Basic - Glazed,Thu,50.0,72.0,False,120.0
1,2020-11-20,Basic - Glazed,Fri,50.0,73.0,False,126.0
2,2020-11-21,Basic - Glazed,Sat,48.0,76.0,False,120.0
3,2020-11-22,Basic - Glazed,Sun,47.0,75.0,False,132.0
4,2020-11-23,Basic - Glazed,Mon,46.0,73.0,False,54.0


The predictions are saved as a .csv in `/content/drive/My Drive/concha_planners/[planner_name]/forecast/` where you can open the file in Google Sheets.

## Updating the Products

The optimal profit point for each product depends on the profit margin and production batch size. You can update those by name.

Each product has a:

   `batch_size`: The number of units made per batch.
    
   `batch_cost`: The marginal cost to produce a batch.
    
   `unit_sale_price`: The sale price of one unit.

In [5]:
# See the products in the transaction history
planner.product()

['Basic - Glazed', 'Basic - Maple Bar', 'Basic - Chocolate', '4', '5']

In [8]:
#@title See a Product's Information

product_name = "" #@param {type:"string"}
planner.product(product_name)

{'batch_size': 6, 'batch_cost': 15.0, 'unit_sale_price': 4.0}

In [7]:
#@title Set a Product's Information

product_name = "" #@param {type:"string"}
batch_size = 1 #@param {type:"integer"}
batch_cost = 2.0 #@param {type:"number"}
unit_sale_price = 4.0 #@param {type:"number"}

planner.product(product_name, batch_size=batch_size, batch_cost=batch_cost, unit_sale_price=unit_sale_price)
planner.product(product_name)

{'batch_size': 6, 'batch_cost': 15.0, 'unit_sale_price': 4.0}

In [ ]:
#@title Limiting Which Products to Predict

# You can control which products are imported to the transaction history.
# Otherwise, every a prediction will be made for every product.

products_list = ['product-1', 'product-2'] #@param {type:"raw"}
planner.update_history(products=products_list)